In [ ]:
# Some useful modules for notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from diffusers import DiffusionPipeline
from scipy import signal
import numpy as np
import torch
import matplotlib.pyplot as plt
import h5py
from tqdne.conf import DATASETDIR
from pathlib import Path
from tqdne.dataset import H5Dataset, RandomDataset
from torch.utils.data import DataLoader
from diffusers import UNet1DModel
from diffusers import DDPMScheduler
from tqdne.diffusers import DDPMPipeline1DCond
from tqdne.lightning import LightningDDMP, LogCallback

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pathlib import Path
from pytorch_lightning.loggers import WandbLogger

from tqdne.conf import OUTPUTDIR, PROJECT_NAME

import pytorch_lightning as pl
import logging


In [ ]:

t = (5501 // 32) * 32
batch_size = 64


path_train = DATASETDIR / Path("data_train.h5")
path_test = DATASETDIR / Path("data_test.h5")
train_dataset = H5Dataset(path_train, cut=t)
test_dataset = H5Dataset(path_test, cut=t)

channels = train_dataset[0][0].shape[0]

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
low_res, high_res = train_dataset[0]

low_res.shape, high_res.shape


In [ ]:
name = "1D-UNET-UPSAMPLE-DDPM"

# Load model
dirpath = OUTPUTDIR / Path(name)

checkpoints = sorted(list(dirpath.glob("*.ckpt")))
# Load the checkpoint with the latest epoch
checkpoint = checkpoints[-1]

logging.info(f"Last checkpoint is : {checkpoint}")
model = LightningDDMP.load_from_checkpoint(checkpoint)


In [ ]:
low_res, high_res = next(iter(test_loader))
reconstructed = model.evaluate(low_res)


In [ ]:
fs = 100
time = np.arange(0, t)/fs

plt.figure(figsize=(6, 3))
plt.plot(time ,low_res[0,0].numpy(), 'b', label="Input")
plt.plot(time, high_res[0,0].numpy(), 'r', label="Target")
plt.xlabel("Time (s)")
plt.xlim(20, 30)
plt.legend()

In [ ]:
b, c, t = reconstructed.shape
fs = 100
time = np.arange(0, t)/fs
i = 0
fig = plt.figure(figsize=(6, 3))
plt.plot(time ,low_res[i,0].cpu().numpy(), 'b', label="Input")
plt.plot(time, high_res[i,0].cpu().numpy(), 'r', label="Target")
plt.plot(time, reconstructed[i,0].cpu().numpy(), 'g', alpha=0.5, label="Reconstructed")
# plt.xlim(1, 5)
plt.xlim(20, 30)

plt.legend()

from tqdne.utils import fig2PIL

image = fig2PIL(fig)




In [ ]:
                # fig = plt.figure(figsize=(6, 3))
                # plt.plot(time, low_res[i, 0].cpu().numpy(), "b", label="Input")
                # plt.plot(time, high_res[i, 0].cpu().numpy(), "r", label="Target")
                # plt.plot(
                #     time,
                #     reconstructed[i, 0].cpu().numpy(),
                #     "g",
                #     alpha=0.5,
                #     label="Reconstructed",
                # )
                # plt.xlim(1, 5)
                # plt.legend()
                # plt.tight_layout()
                # image = fig2PIL(fig)
                # self.wandb_logger.log_image(
                #     key=f"samples {i} - channel {j}", images=[image]
                # )